# Notebook 10: Complete Research Workflow

## From Database to Publication-Ready Results

This notebook integrates everything from the workshop into a **complete, reproducible workflow** for computational materials research.

---

In [ ]:
# =============================================================================
# COMPLETE RESEARCH WORKFLOW - MASTER TEMPLATE
# =============================================================================

import sys
from pathlib import Path
import importlib.util
import re

_cwd = Path.cwd().resolve()
if (_cwd / 'workshop_setup.py').exists():
    sys.path.insert(0, str(_cwd))
elif (_cwd / 'notebooks_enhanced' / 'workshop_setup.py').exists():
    sys.path.insert(0, str(_cwd / 'notebooks_enhanced'))
else:
    raise FileNotFoundError("Cannot find workshop_setup.py. Run from the workshop root or notebooks_enhanced/")

from workshop_setup import *

from ase import Atoms
from ase.build import bulk
from ase.io import read, write
from ase.io.espresso import write_espresso_in
from ase.eos import EquationOfState

from dataclasses import dataclass
from datetime import datetime
from typing import Dict, List, Optional, Tuple

PYMATGEN_AVAILABLE = (importlib.util.find_spec('pymatgen') is not None)
if PYMATGEN_AVAILABLE:
    from pymatgen.core import Structure, Composition
    from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
    print("✓ pymatgen available")
else:
    print("⚠ pymatgen not available")

# =============================================================================
# >>> STUDENT: DEFINE YOUR MATERIAL HERE <<<
# =============================================================================

MATERIAL_NAME = 'Silicon'
FORMULA = 'Si'
STRUCTURE_TYPE = 'diamond'
LATTICE_A = 5.43

FUNCTIONAL = 'PBE'

# =============================================================================
# AUTOMATIC SETUP
# =============================================================================

atoms = bulk(FORMULA, STRUCTURE_TYPE, a=LATTICE_A)
elements = list(set(atoms.get_chemical_symbols()))

pp_files = setup_pseudopotentials(elements, functional=FUNCTIONAL)
ecutwfc, ecutrho = get_recommended_cutoffs(elements, functional=FUNCTIONAL)
PSEUDO_DIR_FUNC = get_pseudo_dir(FUNCTIONAL)

WORK_DIR = OUTPUT_DIR / f'10_workflow_{FORMULA.lower()}'
WORK_DIR.mkdir(parents=True, exist_ok=True)
(WORK_DIR / 'tmp').mkdir(exist_ok=True)

print(f"\n{'='*70}")
print(f"COMPLETE RESEARCH WORKFLOW: {MATERIAL_NAME}")
print(f"{'='*70}")
print(f"Formula:      {FORMULA}")
print(f"Structure:    {STRUCTURE_TYPE}, a = {LATTICE_A} Å")
print(f"Functional:   {FUNCTIONAL}")
print(f"Cutoffs:      ecutwfc={ecutwfc} Ry, ecutrho={ecutrho} Ry")
print(f"Pseudo dir:   {PSEUDO_DIR_FUNC}")
print(f"Output dir:   {WORK_DIR}")
print(f"{'='*70}")
print(f"\nReady to run calculations. Proceed to next cells.")

---

## The Complete DFT Workflow

```
╔═══════════════════════════════════════════════════════════════════════════════╗
║                        RIGOROUS DFT RESEARCH WORKFLOW                         ║
║                     "Garbage In → Garbage Out" Prevention                     ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  ┌─────────────────────────────────────────────────────────────────────────┐  ║
║  │ PHASE 1: STRUCTURE DISCOVERY & VALIDATION                               │  ║
║  ├─────────────────────────────────────────────────────────────────────────┤  ║
║  │                                                                         │  ║
║  │  1.1 Database Search ──────────────────────────────────────────────────│  ║
║  │      • Materials Project, OQMD, AFLOW, JARVIS, NOMAD                   │  ║
║  │      • Identify candidate structures                                    │  ║
║  │      • Check existing calculations (don't reinvent the wheel!)         │  ║
║  │                               ↓                                         │  ║
║  │  1.2 Structure Validation ─────────────────────────────────────────────│  ║
║  │      □ Charge neutrality check                                         │  ║
║  │      □ Bond length reasonableness (Shannon radii)                      │  ║
║  │      □ Space group consistency                                         │  ║
║  │      □ No atomic overlaps                                              │  ║
║  │                               ↓                                         │  ║
║  │  1.3 Structure Prediction (if novel material) ─────────────────────────│  ║
║  │      • CALYPSO, USPEX, AIRSS, XtalOpt                                  │  ║
║  │      • Generate multiple candidates                                     │  ║
║  │                                                                         │  ║
║  └─────────────────────────────────────────────────────────────────────────┘  ║
║                                     ↓                                         ║
║  ┌─────────────────────────────────────────────────────────────────────────┐  ║
║  │ PHASE 2: DFT SETUP & CONVERGENCE                                        │  ║
║  ├─────────────────────────────────────────────────────────────────────────┤  ║
║  │                                                                         │  ║
║  │  2.1 Choose Computational Parameters ──────────────────────────────────│  ║
║  │      • Functional: LDA / GGA-PBE / GGA+U / Hybrid                      │  ║
║  │      • Pseudopotentials: NC / US / PAW (use SSSP!)                     │  ║
║  │                               ↓                                         │  ║
║  │  2.2 Convergence Testing ──────────────────────────────────────────────│  ║
║  │      □ ecutwfc: energy converged to 1 meV/atom                         │  ║
║  │      □ k-points: energy converged to 1 meV/atom                        │  ║
║  │      □ smearing (metals): compare MV, Gaussian, FD                     │  ║
║  │                                                                         │  ║
║  └─────────────────────────────────────────────────────────────────────────┘  ║
║                                     ↓                                         ║
║  ┌─────────────────────────────────────────────────────────────────────────┐  ║
║  │ PHASE 3: STRUCTURE OPTIMIZATION                                         │  ║
║  ├─────────────────────────────────────────────────────────────────────────┤  ║
║  │                                                                         │  ║
║  │  3.1 Full Relaxation ──────────────────────────────────────────────────│  ║
║  │      • vc-relax: cell + positions (if needed)                          │  ║
║  │      • relax: positions only                                           │  ║
║  │      • Target: forces < 10⁻⁴ Ry/Bohr, stress < 0.5 kbar               │  ║
║  │                               ↓                                         │  ║
║  │  3.2 Equation of State (optional) ─────────────────────────────────────│  ║
║  │      • 5-7 volumes around equilibrium                                  │  ║
║  │      • Birch-Murnaghan fit → V₀, E₀, B₀, B₀'                          │  ║
║  │                               ↓                                         │  ║
║  │  3.3 Magnetic Ground State (if magnetic) ──────────────────────────────│  ║
║  │      • Compare FM, AFM, NM configurations                              │  ║
║  │      • Find lowest energy state                                         │  ║
║  │                                                                         │  ║
║  └─────────────────────────────────────────────────────────────────────────┘  ║
║                                     ↓                                         ║
║  ┌─────────────────────────────────────────────────────────────────────────┐  ║
║  │ PHASE 4: STABILITY VERIFICATION (CRITICAL!)                             │  ║
║  ├─────────────────────────────────────────────────────────────────────────┤  ║
║  │                                                                         │  ║
║  │  ⚠️  DO NOT SKIP THIS PHASE! ⚠️                                         │  ║
║  │                                                                         │  ║
║  │  4.1 Thermodynamic Stability ──────────────────────────────────────────│  ║
║  │      □ Formation energy ΔHf < 0                                        │  ║
║  │      □ Convex hull distance Ehull < 25 meV/atom                        │  ║
║  │                               ↓                                         │  ║
║  │  4.2 Dynamic Stability ────────────────────────────────────────────────│  ║
║  │      □ Phonon calculation (ph.x)                                       │  ║
║  │      □ NO imaginary frequencies                                        │  ║
║  │                               ↓                                         │  ║
║  │  4.3 Mechanical Stability ─────────────────────────────────────────────│  ║
║  │      □ Elastic constants calculated                                    │  ║
║  │      □ Born stability criteria satisfied                               │  ║
║  │                                                                         │  ║
║  │  ═══════════════════════════════════════════════════════════════════   │  ║
║  │  ALL PASSED? → Proceed     ANY FAILED? → STOP, structure invalid       │  ║
║  └─────────────────────────────────────────────────────────────────────────┘  ║
║                                     ↓                                         ║
║  ┌─────────────────────────────────────────────────────────────────────────┐  ║
║  │ PHASE 5: PROPERTY CALCULATIONS                                          │  ║
║  ├─────────────────────────────────────────────────────────────────────────┤  ║
║  │                                                                         │  ║
║  │  5.1 Electronic Properties ────────────────────────────────────────────│  ║
║  │      • Band structure (high-symmetry path)                             │  ║
║  │      • Density of states (DOS, PDOS)                                   │  ║
║  │      • Effective masses (for semiconductors)                           │  ║
║  │                               ↓                                         │  ║
║  │  5.2 Optical Properties ───────────────────────────────────────────────│  ║
║  │      • Dielectric function ε(ω)                                        │  ║
║  │      • Absorption spectrum                                              │  ║
║  │      • Refractive index                                                 │  ║
║  │                               ↓                                         │  ║
║  │  5.3 Thermal Properties ───────────────────────────────────────────────│  ║
║  │      • Phonon DOS → specific heat                                      │  ║
║  │      • Thermal conductivity (advanced)                                 │  ║
║  │                               ↓                                         │  ║
║  │  5.4 Transport Properties ─────────────────────────────────────────────│  ║
║  │      • Seebeck coefficient                                              │  ║
║  │      • Electrical conductivity                                         │  ║
║  │                                                                         │  ║
║  └─────────────────────────────────────────────────────────────────────────┘  ║
║                                     ↓                                         ║
║  ┌─────────────────────────────────────────────────────────────────────────┐  ║
║  │ PHASE 6: DOCUMENTATION & REPRODUCIBILITY                                │  ║
║  ├─────────────────────────────────────────────────────────────────────────┤  ║
║  │                                                                         │  ║
║  │  □ Record all computational parameters                                  │  ║
║  │  □ Archive input/output files                                          │  ║
║  │  □ Document software versions                                          │  ║
║  │  □ Create workflow scripts for reproducibility                         │  ║
║  │                                                                         │  ║
║  └─────────────────────────────────────────────────────────────────────────┘  ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
```

---

## Workflow Tracking Structure

In [ ]:
@dataclass
class CalculationRecord:
    """Record of a single calculation."""
    calc_type: str          # 'scf', 'relax', 'bands', 'phonon', etc.
    input_file: str
    output_file: str
    converged: bool
    energy_ry: Optional[float] = None
    notes: str = ""
    timestamp: str = ""

@dataclass
class MaterialWorkflow:
    """Complete workflow tracking for a material."""
    # Identification
    formula: str
    space_group: str
    source: str  # 'Materials Project', 'OQMD', 'predicted', etc.
    
    # Validation status
    charge_neutral: bool = False
    structure_validated: bool = False
    
    # Computational setup
    functional: str = "PBE"
    pseudopotentials: str = "SSSP-efficiency"
    ecutwfc: float = 0.0
    ecutrho: float = 0.0
    kpoints: Tuple[int, int, int] = (1, 1, 1)
    
    # Convergence verified
    ecut_converged: bool = False
    kpts_converged: bool = False
    
    # Structure optimization
    optimized_volume: Optional[float] = None
    bulk_modulus_gpa: Optional[float] = None
    magnetic_config: str = "NM"
    
    # Stability tests
    thermo_stable: Optional[bool] = None
    formation_energy_ev: Optional[float] = None
    ehull_mev: Optional[float] = None
    
    dynamic_stable: Optional[bool] = None
    min_phonon_freq: Optional[float] = None
    
    mechanical_stable: Optional[bool] = None
    
    # Properties
    band_gap_ev: Optional[float] = None
    is_metal: bool = False
    
    # Calculations log
    calculations: List[CalculationRecord] = None
    
    def __post_init__(self):
        if self.calculations is None:
            self.calculations = []
    
    def is_fully_stable(self) -> bool:
        """Check if all stability tests passed."""
        return (self.thermo_stable is True and 
                self.dynamic_stable is True and 
                self.mechanical_stable is True)
    
    def can_calculate_properties(self) -> bool:
        """Check if ready for property calculations."""
        return (self.structure_validated and 
                self.ecut_converged and 
                self.kpts_converged and
                self.is_fully_stable())
    
    def summary(self) -> str:
        """Generate workflow summary."""
        lines = [
            f"Material: {self.formula}",
            f"Space Group: {self.space_group}",
            f"Source: {self.source}",
            "",
            "=== Validation ===",
            f"  Charge neutral: {'PASS' if self.charge_neutral else 'FAIL'}",
            f"  Structure validated: {'PASS' if self.structure_validated else 'PENDING'}",
            "",
            "=== Convergence ===",
            f"  ecutwfc: {self.ecutwfc} Ry {'(converged)' if self.ecut_converged else '(not verified)'}",
            f"  k-points: {self.kpoints} {'(converged)' if self.kpts_converged else '(not verified)'}",
            "",
            "=== Stability ===",
        ]
        
        if self.thermo_stable is not None:
            lines.append(f"  Thermodynamic: {'STABLE' if self.thermo_stable else 'UNSTABLE'}")
            if self.formation_energy_ev is not None:
                lines.append(f"    ΔHf = {self.formation_energy_ev:.3f} eV/atom")
        else:
            lines.append("  Thermodynamic: NOT TESTED")
            
        if self.dynamic_stable is not None:
            lines.append(f"  Dynamic: {'STABLE' if self.dynamic_stable else 'UNSTABLE'}")
        else:
            lines.append("  Dynamic: NOT TESTED")
            
        if self.mechanical_stable is not None:
            lines.append(f"  Mechanical: {'STABLE' if self.mechanical_stable else 'UNSTABLE'}")
        else:
            lines.append("  Mechanical: NOT TESTED")
        
        lines.extend([
            "",
            "=== Properties ===",
        ])
        
        if self.can_calculate_properties():
            if self.band_gap_ev is not None:
                lines.append(f"  Band gap: {self.band_gap_ev:.3f} eV")
            elif self.is_metal:
                lines.append("  Band gap: METAL")
            else:
                lines.append("  Band gap: NOT CALCULATED")
        else:
            lines.append("  NOT READY: Complete stability tests first!")
        
        return "\n".join(lines)

# Example usage
print("Material Workflow Tracking")
print("=" * 60)

In [ ]:
# Example: Create workflow for BaTiO3
bto_workflow = MaterialWorkflow(
    formula="BaTiO3",
    space_group="Pm-3m (221)",
    source="Materials Project (mp-2998)",
    charge_neutral=True,
    structure_validated=True,
    functional="PBE",
    pseudopotentials="SSSP-efficiency",
    ecutwfc=60.0,
    ecutrho=480.0,
    kpoints=(8, 8, 8),
    ecut_converged=True,
    kpts_converged=True,
    thermo_stable=True,
    formation_energy_ev=-3.21,
    ehull_mev=0.0,
    dynamic_stable=True,
    min_phonon_freq=0.5,
    mechanical_stable=True,
    band_gap_ev=2.3,
)

print(bto_workflow.summary())

---

## Master Workflow Script

In [ ]:
# =============================================================================
# COMPLETE WORKFLOW - RUNS ALL PHASES AUTOMATICALLY
# =============================================================================
# This master function chains all calculations together:
#   Phase 1: SCF (ground state)
#   Phase 2: Relaxation (optimize structure)
#   Phase 3: Band structure (electronic properties)
#   Phase 4: DOS (density of states)
#
# Prerequisites are handled automatically - each step checks if the
# previous step was completed before proceeding.
# =============================================================================

def run_complete_workflow(atoms, ecutwfc, prefix='workflow', kpts=(8,8,8)):
    """
    Run complete DFT workflow from structure to properties.
    
    Automatically chains: SCF → Relax → Bands → DOS
    
    Parameters
    ----------
    atoms : ase.Atoms
        Initial structure
    ecutwfc : float
        Wavefunction cutoff
    prefix : str
        Job prefix
    kpts : tuple
        K-point mesh
    
    Returns
    -------
    dict with all results
    """
    results = {
        'formula': atoms.get_chemical_formula(),
        'functional': FUNCTIONAL,
        'phases': {},
    }
    
    elements = list(set(atoms.get_chemical_symbols()))
    pseudopotentials = {el: get_pseudopotential_filename(el, FUNCTIONAL) 
                       for el in elements}
    
    # =========================================================================
    # PHASE 1: SCF
    # =========================================================================
    print("\n" + "=" * 70)
    print("PHASE 1: SCF (Ground State)")
    print("=" * 70)
    
    scf_file = WORK_DIR / f'{prefix}_scf.in'
    input_data = {
        'control': {
            'calculation': 'scf',
            'prefix': prefix,
            'outdir': './tmp',
            'pseudo_dir': str(PSEUDO_DIR_FUNC),
            'tprnfor': True,
            'tstress': True,
        },
        'system': {
            'ecutwfc': ecutwfc,
            'ecutrho': ecutwfc * 8,
            'occupations': 'smearing',
            'smearing': 'cold',
            'degauss': 0.01,
        },
        'electrons': {'conv_thr': 1.0e-8},
    }
    
    write_espresso_in(scf_file, atoms, input_data=input_data,
                      pseudopotentials=pseudopotentials, kpts=kpts)
    
    output, elapsed, converged = run_qe(scf_file, nprocs=NPROCS)
    
    if not converged:
        print("✗ SCF failed! Cannot proceed.")
        results['error'] = 'SCF failed'
        return results
    
    # Parse energy
    for line in output.split('\n'):
        if '!' in line and 'total energy' in line:
            match = re.search(r'=\s+([-\d.]+)\s+Ry', line)
            if match:
                results['phases']['scf'] = {
                    'energy_eV': float(match.group(1)) * RY_TO_EV,
                    'converged': True
                }
    
    print(f"✓ SCF complete. E = {results['phases']['scf']['energy_eV']:.4f} eV")
    
    # =========================================================================
    # PHASE 2: RELAXATION (uses SCF wavefunctions)
    # =========================================================================
    print("\n" + "=" * 70)
    print("PHASE 2: Structure Relaxation")
    print("=" * 70)
    
    relax_file = WORK_DIR / f'{prefix}_relax.in'
    input_data['control']['calculation'] = 'relax'
    input_data['ions'] = {'ion_dynamics': 'bfgs'}
    
    write_espresso_in(relax_file, atoms, input_data=input_data,
                      pseudopotentials=pseudopotentials, kpts=kpts)
    
    output, elapsed, converged = run_qe(relax_file, nprocs=NPROCS, timeout=1800)
    
    results['phases']['relax'] = {'converged': converged}
    if converged:
        print("✓ Relaxation complete")
    else:
        print("⚠ Relaxation may not have fully converged")
    
    # =========================================================================
    # PHASE 3: BAND STRUCTURE (uses SCF charge density)
    # =========================================================================
    print("\n" + "=" * 70)
    print("PHASE 3: Band Structure")
    print("=" * 70)
    
    bands_file = WORK_DIR / f'{prefix}_bands.in'
    input_data['control']['calculation'] = 'bands'
    input_data['control']['verbosity'] = 'high'
    input_data['system']['nbnd'] = len(atoms) * 4
    if 'ions' in input_data:
        del input_data['ions']
    
    write_espresso_in(bands_file, atoms, input_data=input_data,
                      pseudopotentials=pseudopotentials, kpts=kpts)
    
    # Add band path
    with open(bands_file, 'r') as f:
        content = f.read()
    kpath = """K_POINTS {crystal_b}
6
  0.000000 0.000000 0.000000 20  ! G
  0.500000 0.000000 0.500000 10  ! X
  0.500000 0.250000 0.750000 10  ! W
  0.375000 0.375000 0.750000 20  ! K
  0.000000 0.000000 0.000000 20  ! G
  0.500000 0.500000 0.500000 0   ! L
"""
    content = re.sub(r'K_POINTS.*?(?=\n\n|\Z)', kpath, content, flags=re.DOTALL)
    with open(bands_file, 'w') as f:
        f.write(content)
    
    run_qe(bands_file, nprocs=NPROCS)
    results['phases']['bands'] = {'completed': True}
    print("✓ Bands calculation complete")
    
    # =========================================================================
    # PHASE 4: DOS (uses SCF charge density with denser k-mesh)
    # =========================================================================
    print("\n" + "=" * 70)
    print("PHASE 4: Density of States")
    print("=" * 70)
    
    nscf_file = WORK_DIR / f'{prefix}_nscf.in'
    input_data['control']['calculation'] = 'nscf'
    input_data['system']['occupations'] = 'tetrahedra'
    del input_data['system']['smearing']
    del input_data['system']['degauss']
    
    # Denser k-mesh for DOS
    kpts_dos = tuple(k * 2 for k in kpts)
    
    write_espresso_in(nscf_file, atoms, input_data=input_data,
                      pseudopotentials=pseudopotentials, kpts=kpts_dos)
    
    run_qe(nscf_file, nprocs=NPROCS)
    results['phases']['nscf'] = {'completed': True}
    print("✓ NSCF complete")
    
    # =========================================================================
    # SUMMARY
    # =========================================================================
    print("\n" + "=" * 70)
    print("WORKFLOW COMPLETE")
    print("=" * 70)
    print(f"Material: {results['formula']}")
    print(f"Functional: {results['functional']}")
    print(f"\nCompleted phases:")
    for phase, data in results['phases'].items():
        status = "✓" if data.get('converged', data.get('completed', False)) else "⚠"
        print(f"  {status} {phase.upper()}")
    print("=" * 70)
    
    return results


# To run (uncomment):
# results = run_complete_workflow(atoms, ecutwfc)

print("Complete workflow function defined.")
print("\nUsage: results = run_complete_workflow(atoms, ecutwfc)")
print("\nThis chains: SCF → Relax → Bands → DOS")
print("Each phase uses results from previous phases automatically!")

---

## Documentation Template

In [ ]:
def generate_calculation_report(material: MaterialWorkflow) -> str:
    """Generate a documentation report for publication."""
    
    report = f"""
================================================================================
COMPUTATIONAL METHODS - {material.formula}
================================================================================

STRUCTURE SOURCE
----------------
Initial structure obtained from: {material.source}
Space group: {material.space_group}

VALIDATION
----------
- Charge neutrality verified: {'Yes' if material.charge_neutral else 'No'}
- Bond lengths validated against Shannon ionic radii
- Space group symmetry confirmed

COMPUTATIONAL DETAILS
---------------------
All calculations performed with Quantum ESPRESSO v7.x

Exchange-correlation functional: {material.functional}
Pseudopotentials: {material.pseudopotentials}

Convergence parameters (verified to 1 meV/atom):
  - Plane-wave cutoff: {material.ecutwfc} Ry
  - Charge density cutoff: {material.ecutrho} Ry
  - Brillouin zone sampling: {material.kpoints[0]}×{material.kpoints[1]}×{material.kpoints[2]} Monkhorst-Pack grid

Structure optimization:
  - Force convergence: 10⁻⁴ Ry/Bohr
  - Stress convergence: 0.5 kbar
"""
    
    if material.is_fully_stable():
        report += f"""
STABILITY VERIFICATION
----------------------
Thermodynamic stability:
  - Formation energy: {material.formation_energy_ev:.3f} eV/atom
  - Distance from convex hull: {material.ehull_mev:.1f} meV/atom
  - Status: STABLE

Dynamic stability:
  - Phonon calculation performed on 4×4×4 q-grid
  - Minimum phonon frequency: {material.min_phonon_freq:.2f} THz
  - Status: STABLE (no imaginary modes)

Mechanical stability:
  - Born stability criteria: SATISFIED
"""
    
    if material.band_gap_ev is not None:
        report += f"""
ELECTRONIC PROPERTIES
---------------------
Band gap: {material.band_gap_ev:.3f} eV (DFT-{material.functional})

Note: DFT typically underestimates band gaps. For accurate gaps,
consider GW calculations or hybrid functionals.
"""
    
    report += """
DATA AVAILABILITY
-----------------
Input files and calculation outputs are available in the
supplementary materials. All calculations can be reproduced
using the provided workflow scripts.
"""
    
    return report

print(generate_calculation_report(bto_workflow))

---

## Quick Reference Checklist

In [ ]:
def print_workflow_checklist():
    """Print the complete workflow checklist."""
    checklist = """
╔═══════════════════════════════════════════════════════════════╗
║              DFT CALCULATION QUALITY CHECKLIST                ║
╠═══════════════════════════════════════════════════════════════╣
║                                                               ║
║  BEFORE STARTING:                                             ║
║  □ Structure from reliable source                             ║
║  □ Charge neutrality verified                                 ║
║  □ Bond lengths reasonable                                    ║
║  □ Literature search completed                                ║
║                                                               ║
║  DFT SETUP:                                                   ║
║  □ Appropriate functional selected                            ║
║  □ Pseudopotentials from verified library (SSSP)              ║
║  □ ecutwfc convergence tested (< 1 meV/atom)                  ║
║  □ k-points convergence tested (< 1 meV/atom)                 ║
║  □ Smearing appropriate for material type                     ║
║                                                               ║
║  OPTIMIZATION:                                                ║
║  □ Forces converged (< 10⁻⁴ Ry/Bohr)                          ║
║  □ Stress converged (< 0.5 kbar)                              ║
║  □ Final volume reasonable                                    ║
║  □ Magnetic ground state found (if applicable)                ║
║                                                               ║
║  STABILITY (MANDATORY!):                                      ║
║  □ Formation energy calculated                                ║
║  □ Convex hull distance < 25 meV/atom                         ║
║  □ Phonon calculation completed                               ║
║  □ No imaginary frequencies                                   ║
║  □ Elastic constants satisfy Born criteria                    ║
║                                                               ║
║  PROPERTIES:                                                  ║
║  □ Band structure on correct high-symmetry path               ║
║  □ DOS with sufficient k-point density                        ║
║  □ Band gap noted (with DFT limitation caveat)                ║
║                                                               ║
║  DOCUMENTATION:                                               ║
║  □ All parameters recorded                                    ║
║  □ Software versions documented                               ║
║  □ Input/output files archived                                ║
║  □ Results compared with experiment/literature                ║
║                                                               ║
╚═══════════════════════════════════════════════════════════════╝
"""
    print(checklist)

print_workflow_checklist()

---

## Summary

### The Key Lesson

**"Garbage In → Garbage Out"**

No amount of computational sophistication can fix a fundamentally flawed structure or inappropriate methodology.

### Critical Steps (Never Skip!)

1. **Validate your structure** before any calculation
2. **Test convergence** of all computational parameters
3. **Verify stability** before calculating properties
4. **Document everything** for reproducibility

### Workshop Notebooks Summary

| Notebook | Topic | Key Concepts |
|----------|-------|-------------|
| 00 | Overview | Philosophy, workflow, resources |
| 01 | Database Search | MP, OQMD, AFLOW APIs |
| 02 | Structure Validation | Charge neutrality, bond lengths |
| 03 | DFT Setup | Functionals, pseudopotentials |
| 04 | Convergence | ecutwfc, k-points, smearing |
| 05 | Optimization | vc-relax, EOS fitting |
| 06 | Magnetic Systems | FM, AFM, ground state |
| 07 | Stability | Thermodynamic, dynamic, mechanical |
| 08 | Electronic | Bands, DOS, PDOS |
| 09 | Advanced | Optical, phonon, transport |
| 10 | Workflow | Integration, documentation |

### Resources

- **Databases**: Materials Project, OQMD, AFLOW, JARVIS, NOMAD
- **Pseudopotentials**: SSSP (Materials Cloud)
- **Software**: Quantum ESPRESSO, VASP, CASTEP, etc.
- **Visualization**: VESTA, XCrySDen, Materials Studio

---

## Final Notes for Workshop Participants

### Your Next Steps

1. **Practice**: Run the example calculations on your HPC system
2. **Validate**: Always question your inputs and results
3. **Document**: Keep detailed records of all calculations
4. **Compare**: Check your results against literature
5. **Ask**: When in doubt, consult with experienced researchers

### Remember

DFT is a powerful tool, but only when used correctly. The time invested in proper validation and convergence testing will save you from publishing incorrect results.

**Good luck with your research!**